In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE201332"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE201332"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE201332.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE201332.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE201332.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this study involves transcriptional profiling
# in whole blood samples, which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the Sample Characteristics Dictionary:
# Row 1 contains subject status (healthy controls vs MDD patients) -> trait_row
# Row 3 contains age information -> age_row
# Row 2 contains gender information -> gender_row
trait_row = 1
age_row = 3
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary (0 for healthy, 1 for depression)"""
    if not isinstance(value, str):
        return None
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "healthy" in value or "control" in value:
        return 0
    elif "mdd" in value or "depress" in value:
        return 1
    return None

def convert_age(value):
    """Convert age value to continuous integer"""
    if not isinstance(value, str):
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Extract numeric part and remove 'y' (years)
    if 'y' in value:
        try:
            return int(value.replace('y', '').strip())
        except ValueError:
            return None
    try:
        return int(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if not isinstance(value, str):
        return None
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# 3. Save Metadata - Initial Filtering
# trait_row is not None, so trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Create a clinical data DataFrame from the sample characteristics dictionary
    sample_characteristics = {
        0: ['tissue: whole blood'], 
        1: ['subject status: heathy controls', 'subject status: MDD patients'], 
        2: ['gender: male', 'gender: female'], 
        3: ['age: 48y', 'age: 33y', 'age: 43y', 'age: 24y', 'age: 45y', 'age: 36y', 'age: 59y', 
            'age: 51y', 'age: 26y', 'age: 25y', 'age: 32y', 'age: 39y', 'age: 41y', 'age: 52y', 
            'age: 53y', 'age: 44y', 'age: 22y', 'age: 47y', 'age: 54y', 'age: 28y', 'age: 30y']
    }
    
    # Convert the dictionary to a DataFrame format that can be used with geo_select_clinical_features
    clinical_data = pd.DataFrame(sample_characteristics)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    clinical_preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(clinical_preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll implement code specifically to handle GEO compressed files and extract sample characteristics from the series matrix file format.

```python
import os
import pandas as pd
import json
import numpy as np
import gzip
import re
from typing import Callable, Optional, Dict, Any, List, Tuple

# Function to extract sample characteristics from GEO series matrix file
def extract_geo_characteristics(matrix_file_path: str) -> Tuple[Dict[str, List[str]], pd.DataFrame]:
    """Extract sample characteristics and data from a GEO series matrix file."""
    # Initialize variables
    characteristics = {}
    sample_ids = []
    char_indices = {}
    in_characteristics = False
    series_matrix_lines = []
    data_lines = []
    
    # Read the gzipped file
    with gzip.open(matrix_file_path, 'rt') as f:
        for line in f:
            line = line.strip()
            series_matrix_lines.append(line)
            
            # Detect sample characteristics lines
            if line.startswith('!Sample_characteristics_ch'):
                in_characteristics = True
                parts = line.split('\t')
                key = parts[0].replace('!Sample_characteristics_ch', '').strip()
                if key not in char_indices:
                    char_indices[key] = len(char_indices)
                    characteristics[str(char_indices[key])] = []
                
                # Add sample characteristic values for each sample
                for value in parts[1:]:
                    characteristics[str(char_indices[key])].append(value)
            
            # Collect sample IDs
            elif line.startswith('!Sample_geo_accession'):
                sample_ids = line.split('\t')[1:]
            
            # Detect the beginning of data section
            elif line.startswith('!series_matrix_table_begin'):
                in_characteristics = False
                in_data = True
                
            # Collect data lines
            elif in_data and not line.startswith('!series_matrix_table_end'):
                data_lines.append(line)
            
            # End of data section
            elif line.startswith('!series_matrix_table_end'):
                break
    
    # Extract relevant background information for gene availability check
    background_info = "\n".join([line for line in series_matrix_lines 
                                if line.startswith('!Series_summary') or 
                                   line.startswith('!Series_title') or
                                   line.startswith('!Series_type')])
    
    # Create a DataFrame if data section is found
    clinical_data = None
    if data_lines:
        # First line contains column headers
        headers = data_lines[0].split('\t')
        # Data starts from second line
        data = [line.split('\t') for line in data_lines[1:]]
        
        # Create a DataFrame with gene expression data
        gene_data = pd.DataFrame(data, columns=headers)
        
        # Create a transposed version as clinical data
        # Here we assume samples are columns in the original data
        clinical_data = pd.DataFrame(index=sample_ids)
        
    return characteristics, clinical_data, background_info

# Find and process GEO series matrix file
matrix_file = os.path.join(in_cohort_dir, "GSE201332_series_matrix.txt.gz")

if os.path.exists(matrix_file):
    print(f"Found matrix file: {matrix_file}")
    sample_characteristics, clinical_data, background_info = extract_geo_characteristics(matrix_file)
    
    # Print sample characteristics to understand the data structure
    for key, values in sample_characteristics.items():
        if len(values) > 0:
            unique_values = set(values)
            print(f"Key {key}, Example value: {values[0]}")
            print(f"Key {key}, Unique values: {unique_values if len(unique_values) < 5 else list(unique_values)[:5]}")
else:
    print("Matrix file not found!")
    sample_characteristics = {}
    clinical_data = None
    background_info = ""

# 1. Check for gene expression data availability
is_gene_available = True  # Default to True unless we find evidence otherwise

if "miRNA" in background_info and "gene expression" not in background_info.lower():
    is_gene_available = False
if "methylation" in background_info and "gene expression" not in background_info.lower():
    is_gene_available = False

# 2. Variable Availability and Data Type Conversion
trait_row = None
age_row = None
gender_row = None

# Examine sample characteristics to identify rows for trait, age, and gender
if sample_characteristics:
    for key, values in sample_characteristics.items():
        if not values:  # Skip empty lists
            continue
            
        value_str = " ".join(values).lower()
        
        # Look for depression-related indicators in values
        depression_keywords = ["depression", "depressive", "mdd", "major depression", "depressed", "patient: "]
        if any(keyword.lower() in value_str for keyword in depression_keywords):
            trait_row = int(key)
        
        # Look for age indicators
        if any(("age:" in val.lower() or "age :" in val.lower() or "years" in val.lower()) for val in values):
            age_row = int(key)
        
        # Look for gender indicators
        if any(("gender:" in val.lower() or "gender :" in val.lower() or 
               "sex:" in val.lower() or "sex :" in val.lower() or
               "male" in val.lower() or "female" in val.lower()) for val in values):
            gender_row = int(key)

# Define conversion functions
def convert_trait(value: str) -> Optional[int]:
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if "depression" in value or "depressive" in value or "mdd" in value or "patient" in value:
        return 1
    elif "control" in value or "healthy" in value or "non-depression" in value or "normal" in value:
        return 0
    return None

def convert_age(value: str) -> Optional[float]:
    if pd.isna(value) or value is None:
        return None
    
    value = str(value)
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric value (age in years)
    age_match = re.search(r'(\d+(?:\.\d+)?)', value)
    if age_match:
        try:
            return float(age_match.group(1))
        except ValueError:
            return None
    return None

def convert_gender(value: str) -> Optional[int]:
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if "female" in value or "f" == value.strip() or "woman" in value:
        return 0
    elif "male" in value or "m" == value.strip() or "man" in value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

print(f"Gene data available: {is_gene_available}")
print(f"Trait data available: {is_trait_available}")
print(f"Trait row: {trait_row}")
print(f"Age row: {age_row}")
print(f"Gender row: {gender_row}")

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available and clinical data exists, extract clinical features
if trait_row is not


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Task: Dataset Analysis and Clinical Feature Extraction

import os
import pandas as pd
import json
import gzip
from typing import Optional, Callable, Dict, Any

# 1. First, let's examine the available data files in the cohort directory
available_files = os.listdir(in_cohort_dir)
print(f"Available files in the cohort directory: {available_files}")

# Let's load the series matrix file which contains both gene expression and clinical data
matrix_file = os.path.join(in_cohort_dir, 'GSE201332_series_matrix.txt.gz')

# Function to extract sample characteristics from series matrix file
def extract_characteristics(file_path):
    characteristics_data = []
    sample_titles = None
    
    with gzip.open(file_path, 'rt') as f:
        for line in f:
            line = line.strip()
            if line.startswith('!Sample_geo_accession'):
                sample_titles = line.split('\t')[1:]
            elif line.startswith('!Sample_characteristics_ch'):
                parts = line.split('\t')
                row_name = parts[0]
                values = parts[1:]
                characteristics_data.append((row_name, values))
            elif line.startswith('!series_matrix_table_begin'):
                break
    
    # Create DataFrame from characteristics
    if sample_titles and characteristics_data:
        df = pd.DataFrame({i: values for i, (_, values) in enumerate(characteristics_data)})
        df.index = sample_titles
        return df.transpose()
    
    return pd.DataFrame()

# Extract clinical data
clinical_data = extract_characteristics(matrix_file)
print("\nClinical data preview:")
print(clinical_data.head())
print(f"Clinical data shape: {clinical_data.shape}")

# Check if gene expression data exists
is_gene_available = True  # Default assumption based on the file being a series matrix

# Now examine each row to find trait, age, and gender information
row_descriptions = []
for i, row in clinical_data.iterrows():
    unique_values = set(row)
    row_descriptions.append((i, unique_values))
    print(f"Row {i}: {list(unique_values)[:3]}{'...' if len(unique_values) > 3 else ''}")

# Based on the row contents, identify trait_row, age_row, and gender_row
trait_row = None
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    if pd.isna(value):
        return None
    
    value = str(value).lower() if value is not None else ""
    
    # Extract the actual value if there's a colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for depression, 0 for control
    if any(term in value for term in ['depression', 'mdd', 'major depressive disorder']):
        return 1
    elif any(term in value for term in ['control', 'healthy', 'normal']):
        return 0
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    
    value = str(value)
    
    # Extract the actual value if there's a colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    import re
    age_match = re.search(r'(\d+(?:\.\d+)?)', value)
    if age_match:
        return float(age_match.group(1))
    
    return None

def convert_gender(value):
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    
    # Extract the actual value if there's a colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Female: 0, Male: 1
    if any(term in value for term in ['f', 'female', 'women', 'woman']):
        return 0
    elif any(term in value for term in ['m', 'male', 'men', 'man']):
        return 1
    
    return None

# Search for trait, age, and gender rows by examining values
for i, values in row_descriptions:
    values_str = ' '.join([str(v).lower() for v in values])
    
    # Look for depression/MDD indicators
    if trait_row is None and ('depression' in values_str or 'mdd' in values_str or 'control' in values_str):
        trait_row = i
        print(f"Found trait row at index {i}")
    
    # Look for age indicators
    if age_row is None and ('age' in values_str or 'years' in values_str):
        age_row = i
        print(f"Found age row at index {i}")
    
    # Look for gender indicators
    if gender_row is None and ('gender' in values_str or 'sex' in values_str or 'male' in values_str or 'female' in values_str):
        gender_row = i
        print(f"Found gender row at index {i}")

# Check trait availability
is_trait_available = trait_row is not None

# Initial validation to check if this dataset is worth processing further
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available and trait_row is identified, extract and save features
if is_trait_available and clinical_data is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("\nClinical features preview:")
    print(preview)
    
    # Save clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


### Step 5: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 6: Gene Identifier Review

In [ ]:
# Evaluating gene identifiers
# The identifiers shown (1, 2, 3, etc.) are numeric indices, not human gene symbols
# These are likely probe IDs or feature IDs from a microarray or sequencing platform
# They need to be mapped to proper gene symbols for biological interpretation

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


### Step 8: Gene Identifier Mapping

In [ ]:
# Examine gene_annotation to find the column containing gene symbols
print("Gene annotation columns:", gene_annotation.columns.tolist())

# After examining the annotation data and the first few rows, I need to determine which
# columns contain the gene identifiers and gene symbols

# Check if 'GENE_SYMBOL' column exists in the annotation
if 'GENE_SYMBOL' in gene_annotation.columns:
    gene_col = 'GENE_SYMBOL'
elif 'GENE' in gene_annotation.columns:
    gene_col = 'GENE'
elif 'SYMBOL' in gene_annotation.columns:
    gene_col = 'SYMBOL'
else:
    # If no obvious gene symbol column is found, check if gene symbol information
    # might be embedded in another column like NAME or SPOT_ID
    # For Agilent platforms, sometimes gene info is in the SEQUENCE column
    sample_rows = gene_annotation.iloc[100:120]  # Check rows after control probes
    print("Sample rows to look for gene symbols:")
    print(sample_rows[['ID', 'NAME', 'SPOT_ID']].head())
    
    # Default to NAME column which often contains gene information
    gene_col = 'NAME'

# The ID column should be the probe identifier that matches the gene expression data
id_col = 'ID'

print(f"Using {id_col} for probe identifiers and {gene_col} for gene symbols")

# Get mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, id_col, gene_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("Mapping preview:")
print(mapping_df.head(10))

# Apply the mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols:")
print(gene_data.index[:10].tolist())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails
    
# 2. Recreate clinical data using correct row indices from step 2
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features using correct row indices and conversion functions
def convert_trait(value):
    """Convert depression history data to binary format (0 = no, 1 = yes)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if value == 'yes':
        return 1
    elif value == 'no':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age data to continuous format"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        age_str = value.split(':', 1)[1].strip()
    else:
        age_str = value.strip()
    
    try:
        return float(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 = female, 1 = male)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        gender = value.split(':', 1)[1].strip().lower()
    else:
        gender = value.strip().lower()
    
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

# Use correct row indices identified in step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,  # Using the trait variable from context (Depression)
    trait_row=9,  # Using row 9 for depression history as identified in step 2
    convert_trait=convert_trait,
    age_row=1,    # Age data is in row 1
    convert_age=convert_age,
    gender_row=2, # Gender data is in row 2
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from peripheral blood mononuclear cells of older adults with and without depression history, from a study on insomnia disorder."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")